# Audience Resource Thematics
Пример отчета по расчету аудитории интернет-ресурсов (Сross Web) для выбранных тематик ресурсов.

## Описание задачи

Посчитаем аудиторию в разбивке на типы пользования интернетом и ресурсы:
- найдем все тематики ресурсов, которые относятся к агрегаторам ("Агрегатор");
- получим их идентификаторы
- расчитаем отчет

Общие параметры:
- Период: Май 2022
- География: Россия 0+
- Население: 12+
- Тип пользования интернетом: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Mobile)

Статистики:
- Reach (reach)

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Формируем задания

Необходимо сформировать три задания для API Cross Web:
- расчет аудитории ресурса Avito
- расчет совокупной аудитории ресурсов Avito, Ivi
- расчет аудитории Total Internet

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2022-05-01', '2022-05-31')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

### Получим ID тематик ресурсов
Для построения отчета найдем все тематики ресурсов со словом __Агрегатор__.

Для этого воспользуемся методами поиска в медиа справочнике. Работа с медиа справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 


In [ ]:
cats.get_resource_theme(resource_theme='Агрегатор')

Таким образом, необходимые идентификаторы следующие:

- **Агрегаторы товаров и услуг (c2c)**  crossMediaResourceThemeId = 39
- **Агрегаторы товаров и услуг: товарные маркетплейсы**  crossMediaResourceThemeId = 26

## Задания

Перейдем к формированию заданий.


### Задание №1. Расчет аудитории ресурса Avito

In [ ]:
# Задаем название ресурса для отображения в DataFrame
project_name = 'Agg'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Задаем фильтр тематикам ресурсов
mart_filter = 'crossMediaResourceThemeId in (39, 26)'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "useTypeName", "crossMediaResourceName"]

# Указываем список статистик для расчета
statistics = ['reach'] 

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                                      demo_filter, mart_filter, slices, statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_result = mtask.result2table(mtask.get_result(task_audience), project_name = project_name)
df_result

## Сохраняем в Excel

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('audience-agg-resource-thematics'))
df_info = mtask.get_report_info()
df_result.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()